In [35]:
import requests
import pandas as pd
from bs4 import BeautifulSoup

In [36]:
def scrape_holidays(country: str, year: int):
    country = country.strip().lower().replace(" ", "-")
    url = f"https://www.timeanddate.com/holidays/{country}/{year}"

    response = requests.get(url)
    if response.status_code == 404:
        print(f"No holiday data found for {country} in {year}.")
        return

    response.raise_for_status()
    soup = BeautifulSoup(response.content, 'html.parser')
    table = soup.find('table', class_='table--holidaycountry')
    if not table:
        print(f"No holiday table found for {country} in {year}.")
        return

    headers = ['Date', 'Weekday', 'Name', 'Type']
    data = []

    for row in table.find_all('tr'):
        date_cell = row.find('th')
        cells = row.find_all('td')
        if date_cell and len(cells) >= 3:
            raw_date = date_cell.text.strip()
            weekday = cells[0].text.strip()
            name = cells[1].text.strip()
            holiday_type = cells[2].text.strip()
            data.append([raw_date, weekday, name, holiday_type])

    df = pd.DataFrame(data, columns=headers)

    try:
        df['Date'] = pd.to_datetime(df['Date'] + f" {year}")
    except Exception as e:
        print("Date parsing failed:", e)

    filename = f"{country}_holidays_{year}.csv"
    df.to_csv(filename, index=False)
    print(f"Saved: {filename}")
    print(df.head())

# Usage

In [37]:
#scrape_holidays("country", year) (Change the country and year as you need)

# scrape_holidays("austria", 2029)
# scrape_holidays("Australia", 2019)
scrape_holidays("Ghana", 2023)

Saved: ghana_holidays_2023.csv
        Date   Weekday                          Name            Type
0 2023-01-01    Sunday                New Year's Day  Public Holiday
1 2023-01-07  Saturday              Constitution Day  Public Holiday
2 2023-01-09    Monday  Day off for Constitution Day  Public Holiday
3 2023-03-06    Monday              Independence Day  Public Holiday
4 2023-03-20    Monday                 March Equinox          Season
